In [2]:
# import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

In [1]:
from reader import *
from string_matching_by_word import *
from string_matching_by_spaCy_NP import *
from word_embeddings_by_spaCy_NP import *
from hobbs import entry
from nltk import pos_tag

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
file_number = 'b20'

### Read Document

In [4]:
input_path = '/Users/ambuj/Documents/MS Stuff/nlp_cs_6340/final_project/nlp-project2019/dev/{}.input'.format(file_number)
key_path = '/Users/ambuj/Documents/MS Stuff/nlp_cs_6340/final_project/nlp-project2019/dev/{}.key'.format(file_number)

In [5]:
read = ReadInput(input_path)
list_of_sentences = read.getListOfSentences()
full_text = read.getText()

### Get Sentence Dict and Cluster Head Dict

In [6]:
sentence_dict = getSentenceDict(list_of_sentences)
cluster_head_dict = getClusterHeads(sentence_dict)

### Get Coref for exact match by word

In [7]:
coref_dict_match_word = getCorefDict_match_word(sentence_dict, cluster_head_dict,90)
# coref_dict_match_word

### Get Coref for threshold match by NP

In [8]:
coref_dict_match_NP = getCorefDict_match_NP(sentence_dict, cluster_head_dict,80)
# coref_dict_match_NP

### Get Coref for Word Embedding Similarity by NP

In [9]:
# coref_dict_all_sorted = getCorefDict_all_sorted(sentence_dict, cluster_head_dict,0.5)
# coref_dict_max_sentence = getCorefDict_max_of_each_sentence(sentence_dict, cluster_head_dict,0.5)

In [10]:
# coref_dict_all_sorted_top3 = get_TopN_Matches(coref_dict_all_sorted,3)

### Merging dictionaries

In [11]:
def mergeDicts(dict1, dict2):
    
    d1 = dict1.copy()
    d2 = dict2.copy()
    
    d1_keys = d1.keys()
    d2_keys = d2.keys()
    
    for key in d1_keys:
        d1[key] = list(map(lambda x: x[0:3], d1[key]))
    
    for key in d2_keys:
        d2[key] = list(map(lambda x: x[0:3], d2[key]))
    
    for key in d2_keys:
        if(key not in d1_keys):
            d1[key] = d2[key]
        else:
            for val in d2[key]:
                if(val not in d1[key]):
                    d1[key].append(val)
    
    return dict(sorted(d1.items()))

In [12]:
coref_final = mergeDicts(coref_dict_match_word, coref_dict_match_NP)

In [13]:
coref_final

{'X0': [['OPEC', 12, 'OPEC'],
  ['OPEC', 14, 'OPEC'],
  ['OPEC', 17, 'OPEC'],
  ['OPEC', 18, 'OPEC'],
  ['OPEC', 19, 'OPEC'],
  ['OPEC', 22, 'OPEC'],
  ['OPEC', 23, 'OPEC'],
  ['OPEC', 24, 'OPEC'],
  ['OPEC', 25, 'OPEC']],
 'X1': [[a Price War, 5, 'a possible price war'],
  [a Price War, 19, 'its war']],
 'X10': [[Saudi Arabia, 23, 'Saudi Arabia']],
 'X11': [[its government, 14, 'the government']],
 'X12': [[conciliatory gestures to OPEC, 10, 'The gestures']],
 'X13': [['yesterday', 14, 'yesterday'], ['yesterday', 21, 'yesterday']],
 'X14': [[Abu Dhabi, 20, 'Abu Dhabi']],
 'X18': [[its assigned quota of 948,000 barrels a day,
   14,
   'a higher OPEC quota'],
  [its assigned quota of 948,000 barrels a day, 17, 'their quota'],
  [its assigned quota of 948,000 barrels a day,
   20,
   "the Emirates' unilateral quota decision"]],
 'X2': [['Output', 6, 'output'],
  ['Output', 18, 'output'],
  ['Output', 23, 'output'],
  ['Output', 24, 'output']],
 'X20': [[the Emirates' official news agenc

### Passing the reference dict to Hobbs

In [14]:
coref_final_with_pro = entry(list_of_sentences, cluster_head_dict, coref_final)

In [15]:
coref_final_with_pro

{'X0': [['OPEC', 12, 'OPEC'],
  ['OPEC', 14, 'OPEC'],
  ['OPEC', 17, 'OPEC'],
  ['OPEC', 18, 'OPEC'],
  ['OPEC', 19, 'OPEC'],
  ['OPEC', 22, 'OPEC'],
  ['OPEC', 23, 'OPEC'],
  ['OPEC', 24, 'OPEC'],
  ['OPEC', 25, 'OPEC']],
 'X1': [[a Price War, 5, 'a possible price war'],
  [a Price War, 19, 'its war']],
 'X10': [[Saudi Arabia, 23, 'Saudi Arabia']],
 'X11': [[its government, 14, 'the government']],
 'X12': [[conciliatory gestures to OPEC, 10, 'The gestures']],
 'X13': [['yesterday', 14, 'yesterday'], ['yesterday', 21, 'yesterday']],
 'X14': [[Abu Dhabi, 20, 'Abu Dhabi']],
 'X18': [[its assigned quota of 948,000 barrels a day,
   14,
   'a higher OPEC quota'],
  [its assigned quota of 948,000 barrels a day, 17, 'their quota'],
  [its assigned quota of 948,000 barrels a day,
   20,
   "the Emirates' unilateral quota decision"]],
 'X2': [['Output', 6, 'output'],
  ['Output', 18, 'output'],
  ['Output', 23, 'output'],
  ['Output', 24, 'output']],
 'X20': [[the Emirates' official news agenc

### Adding those Cluster Heads which were not predicted

In [16]:
def addNOPredsClusterHeads(cluster_head_dict, coref_final_with_pro):
    for cluster_id, cluster_val in cluster_head_dict.items():
        if(cluster_id not in coref_final_with_pro.keys()):
            coref_final_with_pro[cluster_id] = [['',int(cluster_val[0]),'', None]]
            
    return coref_final_with_pro

In [17]:
coref_final_with_pro = addNOPredsClusterHeads(cluster_head_dict, coref_final_with_pro)

In [18]:
coref_final_with_pro

{'X0': [['OPEC', 12, 'OPEC'],
  ['OPEC', 14, 'OPEC'],
  ['OPEC', 17, 'OPEC'],
  ['OPEC', 18, 'OPEC'],
  ['OPEC', 19, 'OPEC'],
  ['OPEC', 22, 'OPEC'],
  ['OPEC', 23, 'OPEC'],
  ['OPEC', 24, 'OPEC'],
  ['OPEC', 25, 'OPEC']],
 'X1': [[a Price War, 5, 'a possible price war'],
  [a Price War, 19, 'its war']],
 'X10': [[Saudi Arabia, 23, 'Saudi Arabia']],
 'X11': [[its government, 14, 'the government']],
 'X12': [[conciliatory gestures to OPEC, 10, 'The gestures']],
 'X13': [['yesterday', 14, 'yesterday'], ['yesterday', 21, 'yesterday']],
 'X14': [[Abu Dhabi, 20, 'Abu Dhabi']],
 'X18': [[its assigned quota of 948,000 barrels a day,
   14,
   'a higher OPEC quota'],
  [its assigned quota of 948,000 barrels a day, 17, 'their quota'],
  [its assigned quota of 948,000 barrels a day,
   20,
   "the Emirates' unilateral quota decision"]],
 'X2': [['Output', 6, 'output'],
  ['Output', 18, 'output'],
  ['Output', 23, 'output'],
  ['Output', 24, 'output']],
 'X20': [[the Emirates' official news agenc

### Taking only the head nouns

In [19]:
def takeHeadNouns(coref_final_with_pro):
    
    for cluster in coref_final_with_pro.keys():
        for i in range(0, len(coref_final_with_pro[cluster])):
            coref_final_with_pro[cluster][i][2] = coref_final_with_pro[cluster][i][2].split(' ')[-1]
                        
    return coref_final_with_pro

In [20]:
coref_final_with_pro = takeHeadNouns(coref_final_with_pro)

In [21]:
coref_final_with_pro

{'X0': [['OPEC', 12, 'OPEC'],
  ['OPEC', 14, 'OPEC'],
  ['OPEC', 17, 'OPEC'],
  ['OPEC', 18, 'OPEC'],
  ['OPEC', 19, 'OPEC'],
  ['OPEC', 22, 'OPEC'],
  ['OPEC', 23, 'OPEC'],
  ['OPEC', 24, 'OPEC'],
  ['OPEC', 25, 'OPEC']],
 'X1': [[a Price War, 5, 'war'], [a Price War, 19, 'war']],
 'X10': [[Saudi Arabia, 23, 'Arabia']],
 'X11': [[its government, 14, 'government']],
 'X12': [[conciliatory gestures to OPEC, 10, 'gestures']],
 'X13': [['yesterday', 14, 'yesterday'], ['yesterday', 21, 'yesterday']],
 'X14': [[Abu Dhabi, 20, 'Dhabi']],
 'X18': [[its assigned quota of 948,000 barrels a day, 14, 'quota'],
  [its assigned quota of 948,000 barrels a day, 17, 'quota'],
  [its assigned quota of 948,000 barrels a day, 20, 'decision']],
 'X2': [['Output', 6, 'output'],
  ['Output', 18, 'output'],
  ['Output', 23, 'output'],
  ['Output', 24, 'output']],
 'X20': [[the Emirates' official news agency, 13, 'report']],
 'X21': [['oil', 20, 'oil'],
  ['oil', 20, 'oil'],
  ['oil', 25, 'oil'],
  ['oil', 25

### Print Output

In [22]:
def printOP(cluster_head_dict, coref_final_with_pro):
    
    for cluster_id, cluster_head_name in cluster_head_dict.items():

        print('<COREF ID="{}">{}</COREF>'.format(cluster_id, cluster_head_name[1]))

        coreferences = coref_final_with_pro[cluster_id]
        list_of_sent_ids = list(map(lambda x: x[1], coreferences))
        sorted_index_sent_ids = [i[0] for i in sorted(enumerate(list_of_sent_ids), key=lambda x:x[1])]
        coreferences = [coreferences[i] for i in sorted_index_sent_ids]

        for coref in coreferences:
            if(coref[0] == ''):
                continue
            print('{{{0}}}'.format(coref[1]) + ' ' + '{' + coref[2] + '}')
        print('\n', end = '')      

In [23]:
# %%capture cap --no-stderr
printOP(cluster_head_dict, coref_final_with_pro)

<COREF ID="X0">OPEC</COREF>
{12} {OPEC}
{14} {OPEC}
{17} {OPEC}
{18} {OPEC}
{19} {OPEC}
{22} {OPEC}
{23} {OPEC}
{24} {OPEC}
{25} {OPEC}

<COREF ID="X1">a Price War</COREF>
{5} {war}
{19} {war}

<COREF ID="X2">Output</COREF>
{6} {output}
{18} {output}
{23} {output}
{24} {output}

<COREF ID="X3">Quota Violator</COREF>
{7} {violator}

<COREF ID="X4">price</COREF>
{20} {prices}

<COREF ID="X5">a sharp rise in OPEC production</COREF>

<COREF ID="X6">a day</COREF>
{26} {day}

<COREF ID="X7">20 million barrels a day early this month</COREF>
{6} {barrels}
{15} {barrels}
{24} {barrels}
{26} {barrels}

<COREF ID="X8">its own output</COREF>
{18} {output}
{23} {output}
{24} {output}

<COREF ID="X9">a targeted 1.5 million barrels a day</COREF>
{15} {barrels}
{24} {barrels}
{26} {barrels}

<COREF ID="X10">Saudi Arabia</COREF>
{23} {Arabia}

<COREF ID="X11">its government</COREF>
{14} {government}

<COREF ID="X12">conciliatory gestures to OPEC</COREF>
{10} {gestures}

<COREF ID="X13">yesterday</COREF

In [24]:
# with open('{}.response'.format(file_number),'w') as f:
#     f.write(cap.stdout)

### Answer Key

In [25]:
# reader_o = ReadInput(key_path)
# ans = reader_o.getListOfSentences()
# ans